In [1]:
import numpy
import pandas

import gensim
from gensim.corpora import Dictionary
from gensim.models import TfidfModel

from pythainlp.tokenize import word_tokenize
import pythainlp

from sklearn.cluster import KMeans

import re
import math

from data_tokenizer import load_corpus

### Load Data

In [2]:
file_name = 'ผู้บริโภค - TescoLotus.txt'

corpus, labels = load_corpus('../data/facebook/' + file_name)

len_corpus = len(corpus)
print('Total documents', len_corpus)

clusters = list(set(labels))
print(len(clusters), 'clusters')

f = open('../data/facebook/tokenized/tokenized_' + file_name)
tokenized_corpus = eval(f.read())
f.close()

Total documents 269
1 clusters


### Preprocess Corpus

#### Remove Words

In [3]:
dictionary = Dictionary(tokenized_corpus)
print('origin:', len(dictionary), 'words')

appear_rate = 1
dictionary.filter_extremes(no_below=2, no_above=0.7, keep_n=len(dictionary))
print('filter frequent words:', len(dictionary), 'words')

letter_words = []
for word in dictionary.keys():
    if len(dictionary[word]) <= 1:
        letter_words.append(word)
dictionary.filter_tokens(bad_ids=letter_words)
print('filter letter words:', len(dictionary), 'words')

stopwords = pythainlp.corpus.stopwords.words('thai')
stopwords.append('นี้')
dictionary.add_documents([stopwords])
stopwords = [dictionary.token2id[word] for word in stopwords]
dictionary.filter_tokens(bad_ids=stopwords)
print('filter stop words:', len(dictionary), 'words')

origin: 1313 words
filter frequent words: 540 words
filter letter words: 539 words
filter stop words: 352 words


In [4]:
# bow_corpus = [dictionary.doc2bow(doc) for doc in tokenized_corpus]
idx_corpus = [dictionary.doc2idx(doc) for doc in tokenized_corpus]

temp_corpus = []
for doc in idx_corpus:
    temp_corpus.append([dictionary[id] for id in doc if id >= 0])
    print(temp_corpus[-1])
idx_corpus = temp_corpus

['พนักงาน', 'พนักงาน', 'โลตัส', 'โลตัส', 'ทำ', 'เอ็กเพลส', 'พนักงาน', 'คน', 'ทำ', 'ขาย', 'คน', 'คน', 'เวลา', 'กิน', 'ข้าว', 'ผม', 'คน', 'พนักงาน', 'เหมือน', 'พนักงาน', 'คน', 'ผม', 'รู้', 'พนักงาน', 'ตั้งใจ', 'ทำ', 'งาน', 'คน']
['เลิก', 'ข้าว', 'ดี', 'อี', 'เครื่อง', 'น้ำ']
['ถุง', 'งง', 'ถุง', 'กก', 'เจอ', 'น้ำยา', 'ซัก', 'ผ้า', 'น้ำ', 'ยา', 'ผ้า', 'นุ่ม']
['ถุง', 'แถม', 'ลด', 'ลอง', 'ดู', 'ถุง', 'หนา']
['ทำ', 'เดือน', 'พนักงาน', 'คน', 'คน', 'คน', 'นึง', 'พัน', 'เช้า', 'เย็น']
['อย่า', 'บังคับ', 'น้อง', 'เชียร์', 'อย่า', 'งี่', 'เง่า', 'กี่', 'ปี']
['โลตัส', 'โลตัส', 'เอกเพรส', 'แม่ง', 'เกลียด', 'พนักงาน']
['คน', 'คน', 'คน', 'ดี', 'คน', 'ดี', 'ผม', 'โลก', 'ผม', 'ทำ', 'งาน', 'บริการ', 'เหมือน', 'ยืน', 'จุด', 'ยืน', 'รู้', 'พนักงาน', 'เหมือน', 'ปรับปรุง', 'ดี', 'ปรับปรุง', 'ตัว', 'ดี', 'ดี']
['พนักงาน', 'ทำ', 'หน้า', 'ตูด', 'คน', 'เหมือน', 'บังคับ', 'ทำ', 'งาน', 'งั้น']
['เอ็กเพรส', 'อบรม', 'พนักงาน', 'ป่าว', 'อารม', 'ถาม', 'ราคา', 'ทำ', 'หน้า', 'พอใจ', 'ติด', 'ราคา', 'ถาม', 'หรอก', 'ราค

In [5]:
unique_words = [dictionary[id] for id in range(len(dictionary))]

array = numpy.zeros((len_corpus, len(unique_words)), dtype=int)
for i, vector in enumerate(idx_corpus):
    for word in vector:
        array[i, dictionary.token2id[word]] += 1

word_vectors = pandas.DataFrame(array, columns=unique_words, dtype=int)
word_vectors.head()

,กิน,ขาย,ข้าว,คน,งาน,ตั้งใจ,ทำ,ผม,พนักงาน,รู้,...,ฉีด,ล้อ,สะอาด,ร้อน,แจก,เขียว,พลังงาน,แอร์ไม่,สลิป,นวมินทร์
0,1,1,1,6,1,1,3,2,6,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,3,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0


### K-Mean

In [6]:
kmeans = KMeans(n_clusters=5).fit(word_vectors)
predicted_labels = kmeans.labels_

In [7]:
result = pandas.DataFrame()
result['comment'] = corpus
result['tokenized_comment'] = idx_corpus
result['label'] = labels
result['predicted_label'] = predicted_labels
result.head()

,comment,tokenized_comment,label,predicted_label
0,บางครั้งพนักงานโลตัสเอ็กเพลส กับพนักงานโลตัสให...,"[พนักงาน, พนักงาน, โลตัส, โลตัส, ทำ, เอ็กเพลส,...",,1
1,เลิกเปิดเพลง ข้าวแสนดี กับอีเครื่องกรองน้ำเพีย...,"[เลิก, ข้าว, ดี, อี, เครื่อง, น้ำ]",,0
2,ถุงงง ถุงมึงบอบบางมากกก ยิ่งเจอน้ำยาซักผ้า น้...,"[ถุง, งง, ถุง, กก, เจอ, น้ำยา, ซัก, ผ้า, น้ำ, ...",,0
3,เราไม่เคยรับถุงเลย แถมได้แต้มกรีนพ้อยไว้ใช้เป็...,"[ถุง, แถม, ลด, ลอง, ดู, ถุง, หนา]",,0
4,เคยไปทำ เอ้กซ์เพรส เดือนเดียว พอเลย พนักงานต่...,"[ทำ, เดือน, พนักงาน, คน, คน, คน, นึง, พัน, เช้...",,3


In [8]:
label_count = numpy.unique(predicted_labels, return_counts=True) 
print(label_count)

for cluster in clusters:
    print('\t' + cluster, end='')
print('\tpercent')

for label in range(len(clusters)):
    print(str(label) + '  |', end='')
    
    num_max = 0
    for cluster in clusters:
        loc = result[(result['label'] == cluster) & (result['predicted_label'] == label)]
        if len(loc) > num_max:
            num_max = len(loc)
        print('\t' + str(len(loc)), end='')
    
    print('\t' + str(num_max / label_count[1][label]))

(array([0, 1, 2, 3, 4], dtype=int32), array([211,   2,   5,  31,  20]))
		percent
0  |	211	1.0


In [9]:
for index, value in result[result['predicted_label'] == 4]['comment'].iteritems():
    print(index, '[', value, ']')

13 [ ราคาสินค้า ตอนโปรโมชั่น บางอันหาไม่เจอ เหมือนพนักงานเอาไปซ่อน ให้ลูกค้าวิ่งไล่หา ไอ้ซั้ซไม่ใช่มอญซ่อนผ้า ปลรปภ บางสาขา หน้าตาดุมาก เวลาเข้าห้างนึกว่าทำไรผิดตลอดเวลา ]
24 [ โลตัสเอ็กซ์เพลสสาขายางตลาด จกาฬสินธุ์ค่ะแย่มากวันนั้นไปซื้อแพมเพิสให้ลูกยืนรอคิดเงินมันไม่คิดให้ทั้งที่ไม่มีลุกค้ามันยืนคุยกันสองคนหยอกล้อไปมามองหน้าเราเฉยเลยไม่พุดอะไรสักคำยืนรอ3นาทีได้ค่ะตอนนั้นโมโหมากว่าจะด่าเหมือนกันเลยไม่ทำ  เดินมาอีกช่องนึง ]
36 [ สาขามหาชัยคลองครุ พนงแคชเชียร์ชอบคุยกันข้ามหัวลูกค้า แถมคิดเงินผิดโดยการยิงสินค้าซ้ำกัน 2 ครั้ง ดีนะที่เช็คบิลล์ใบเสร็จก่อนเดินออกจากห้าง นี้เลยรีบไปที่จุดบริการเพื่อทำการรีฟันเงินคืน และต่อว่าพนงแคชเชียร์หน่อยนึง ปล คิดเงินเกินกับสาขานี้มา 3 ครั้งแล้ว ]
37 [ สาขาเล็กแถวหลังตลาดบางใหญ่ พนักงานคิดเงินกะกลางคืน ยังกะเมายามาคิดเงิน คำพูดก็กวนตีนลูกค้า ]
44 [ ช่องจ่ายเงินด่วนสินค้าม่เกิน10ชิ้น คือ อย่ามีเรย เห็นเข็นมาจ่ายเต็มรถเข็นตลอด พนักงานท้วงติงสักนิด ก้อไม่มีห่วยมาก รอคิวเป็นชาติ ช่องเยอะแยะเปิด แค่3 ช่องกุจะบ้า ]
50 [ ป้ายราคาบอกราคาหนึ่งไปจ่ายอีกราคาหนึ่ง เช่

In [10]:
for index, value in result[result['predicted_label'] == 4]['tokenized_comment'].iteritems():
    print(index, value)

13 ['ราคา', 'สินค้า', 'ตอน', 'โปรโมชั่น', 'หา', 'เจอ', 'เหมือน', 'พนักงาน', 'ซ่อน', 'ลูกค้า', 'วิ่ง', 'ไล่', 'หา', 'ไอ้', 'ซ่อน', 'ผ้า', 'หน้าตา', 'เวลา', 'ห้าง', 'นึก', 'เวลา']
24 ['สาขา', 'แย่', 'ซื้อ', 'ลูก', 'ยืน', 'รอ', 'เงิน', 'ยืน', 'คุย', 'สอง', 'คน', 'หน้า', 'สัก', 'ยืน', 'รอ', 'นาที', 'ตอน', 'ด่า', 'เหมือน', 'ทำ', 'เดิน']
36 ['ชอบ', 'คุย', 'ข้าม', 'หัว', 'ลูกค้า', 'แถม', 'เงิน', 'ยิง', 'สินค้า', 'ดี', 'เช็ค', 'ใบ', 'เดิน', 'ห้าง', 'รีบ', 'จุด', 'บริการ', 'ทำ', 'เงิน', 'คืน', 'นึง', 'เงิน', 'สาขา']
37 ['สาขา', 'แถว', 'ตลาด', 'พนักงาน', 'เงิน', 'กะ', 'คืน', 'ยา', 'เงิน', 'กวน', 'ตีน', 'ลูกค้า']
44 ['ช่อง', 'จ่าย', 'เงิน', 'ด่วน', 'สินค้า', 'ชิ้น', 'อย่า', 'เข็น', 'จ่าย', 'เต็ม', 'รถ', 'เข็น', 'พนักงาน', 'สัก', 'ก้อ', 'ห่วย', 'รอ', 'คิว', 'ชาติ', 'ช่อง', 'ช่อง', 'กุ', 'บ้า']
50 ['ป้าย', 'ราคา', 'ราคา', 'จ่าย', 'ราคา', 'ซื้อ', 'แถม', 'พนักงาน', 'เหมือน', 'ด่า', 'สิบ', 'รอ', 'ลด']
56 ['งง', 'ใบ', 'แถม', 'ลด', 'ถุง', 'โปรโมชั่น', 'ลดท้าย', 'ลูกค้า', 'เวลา', 'รอ', 'คิว', 'จ่าย', 'เง

### Dimension Reduction

In [11]:
average_doc_size = 0
for doc in idx_corpus:
    average_doc_size += len(doc)
average_doc_size /= len(idx_corpus)
average_doc_size = math.ceil(average_doc_size)
average_doc_size

word_freq = dictionary.dfs
filtered_corpus = []
for doc in idx_corpus:
    new_doc = [(word, word_freq[dictionary.token2id[word]]) for word in doc]
    new_doc.sort(reverse=True, key=lambda x: x[1])
    new_doc = new_doc[:average_doc_size]
    filtered_corpus.append([word for word, df in new_doc])

In [12]:
dictionary = Dictionary(filtered_corpus)

# dictionary.filter_extremes(no_below=2, no_above=1, keep_n=len(dictionary))
# print(len(dictionary))

unique_words = [dictionary[id] for id in range(len(dictionary))]

array = numpy.zeros((len_corpus, len(unique_words)), dtype=int)
for i, vector in enumerate(filtered_corpus):
    for word in vector:
        array[i, dictionary.token2id[word]] += 1

word_vectors = pandas.DataFrame(array, columns=unique_words, dtype=int)
word_vectors.head()

,ทำ,พนักงาน,ข้าว,ดี,น้ำ,อี,เครื่อง,เลิก,กก,งง,...,พลังงาน,คอย,แอร์ไม่,สลิป,คู,ปอง,นวมินทร์,สิบ,ห้า,แจก
0,3,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
kmeans = KMeans(n_clusters=5).fit(word_vectors)
predicted_labels = kmeans.labels_

In [14]:
result = pandas.DataFrame()
result['comment'] = corpus
result['tokenized_comment'] = filtered_corpus
result['label'] = labels
result['predicted_label'] = predicted_labels
result.head()

,comment,tokenized_comment,label,predicted_label
0,บางครั้งพนักงานโลตัสเอ็กเพลส กับพนักงานโลตัสให...,"[พนักงาน, พนักงาน, พนักงาน, พนักงาน, พนักงาน, ...",,1
1,เลิกเปิดเพลง ข้าวแสนดี กับอีเครื่องกรองน้ำเพีย...,"[ดี, เลิก, น้ำ, เครื่อง, ข้าว, อี]",,0
2,ถุงงง ถุงมึงบอบบางมากกก ยิ่งเจอน้ำยาซักผ้า น้...,"[ถุง, ถุง, เจอ, น้ำ, งง, น้ำยา, ผ้า, ผ้า, กก]",,3
3,เราไม่เคยรับถุงเลย แถมได้แต้มกรีนพ้อยไว้ใช้เป็...,"[ถุง, ถุง, ลด, แถม, ดู, หนา, ลอง]",,3
4,เคยไปทำ เอ้กซ์เพรส เดือนเดียว พอเลย พนักงานต่...,"[พนักงาน, ทำ, คน, คน, คน, นึง, เดือน, เย็น, เช้า]",,2


In [15]:
label_count = numpy.unique(predicted_labels, return_counts=True) 
print(label_count)

for cluster in clusters:
    print('\t' + cluster, end='')
print('\tpercent')

for label in range(len(clusters)):
    print(str(label) + '  |', end='')
    
    num_max = 0
    for cluster in clusters:
        loc = result[(result['label'] == cluster) & (result['predicted_label'] == label)]
        if len(loc) > num_max:
            num_max = len(loc)
        print('\t' + str(len(loc)), end='')
    
    print('\t' + str(num_max / label_count[1][label]))

(array([0, 1, 2, 3, 4], dtype=int32), array([193,  21,  14,  15,  26]))
		percent
0  |	193	1.0


In [20]:
for index, value in result[result['predicted_label'] == 2]['comment'].iteritems():
    print(index, '[', value, ']')

4 [ เคยไปทำ เอ้กซ์เพรส เดือนเดียว พอเลย  พนักงานต่อ1ช่วงกะงานน้อยไป  3คน  บางวัน2 คน  คนเข้าวันนึงเกือบพัน ยิ่งช่วงเช้าเย็นเร่งด่วน วุ่นวายจัด ]
7 [ ทุกคนล้วนแล้ว แต่มีหน้าที่ของเขา คนเราควรให้โอกาสซึ่งกันและกันนะครับให้คิดว่าคนดีก็มี คนไม่ดีก็มีนะครับ ผมไม่ได้โลกสวยนะครับ แต่ผมก็ทำงานในด้านบริการเหมือนกัน ถ้าไม่มายืนตรงจุดที่เขายืนคุณก็จะไม่รู้ คิดบวกนะครับ พนักงานเหมือนกันนะครับ ปรับปรุงตรงไหนแล้วดี ก็ควรที่ต้องปรับปรุงแก้ไขส่วนที่ผิดพลาดของตัวเรา ดีแล้วยังดีได้อีกนะครับ ]
8 [ พนักงานทำหน้าเป็นตูดเกือบทุกคน ประมาณว่าเหมือนถูกบังคับให้มาทำงานยังงั้นแหละ เห็นแล้วพลอย เครียดไปด้วย ]
24 [ โลตัสเอ็กซ์เพลสสาขายางตลาด จกาฬสินธุ์ค่ะแย่มากวันนั้นไปซื้อแพมเพิสให้ลูกยืนรอคิดเงินมันไม่คิดให้ทั้งที่ไม่มีลุกค้ามันยืนคุยกันสองคนหยอกล้อไปมามองหน้าเราเฉยเลยไม่พุดอะไรสักคำยืนรอ3นาทีได้ค่ะตอนนั้นโมโหมากว่าจะด่าเหมือนกันเลยไม่ทำ  เดินมาอีกช่องนึง ]
28 [ พนักงานเหมือนไม่อยากทำงาน หน้าหงิกหน้างอ แต่ก็มีบางคนที่น่ารัก พูดดีส่วนน้อย ]
33 [ เราก็ไม่รู้ระบบการทำงานของเค้านะ แต่เวลาไปซื้อของชอบเอาของมาเติม ลาก

In [22]:
for index, value in result[result['predicted_label'] == 4]['tokenized_comment'].iteritems():
    print(index, value)

25 ['พนักงาน', 'สาขา', 'ลูกค้า', 'หน้า', 'เงิน', 'เจอ', 'เดิน', 'หา', 'ขาย']
30 ['ทำ', 'เงิน', 'คน', 'จ่าย', 'ช่อง', 'เสาร์', 'อาทิตย์', 'โคตร']
35 ['สาขา', 'ทำ', 'เงิน', 'คน', 'จ่าย', 'ช่อง', 'แถว']
36 ['สาขา', 'ลูกค้า', 'ทำ', 'เงิน', 'เงิน', 'เงิน', 'ดี', 'บริการ', 'สินค้า']
37 ['พนักงาน', 'สาขา', 'ลูกค้า', 'เงิน', 'เงิน', 'กะ', 'แถว', 'คืน', 'ตีน']
44 ['พนักงาน', 'เงิน', 'จ่าย', 'จ่าย', 'รถ', 'สินค้า', 'รอ', 'เข็น', 'เข็น']
45 ['พนักงาน', 'พนักงาน', 'เงิน', 'มือ', 'เต็มใจ']
56 ['ลูกค้า', 'เงิน', 'ถุง', 'จ่าย', 'เวลา', 'ลด', 'รอ', 'ใบ', 'แถม']
68 ['พนักงาน', 'พนักงาน', 'โลตัส', 'เงิน', 'จ่าย', 'เลิก', 'ช่อง', 'ช่อง', 'สลับ']
74 ['พนักงาน', 'ลูกค้า', 'ลูกค้า', 'โลตัส', 'เงิน', 'เงิน', 'ซื้อ', 'บริการ', 'สินค้า']
77 ['หน้า', 'เงิน', 'เหมือน', 'จ่าย', 'จ่าย', 'พนง', 'เดือน', 'ตัง', 'ตัง']
80 ['พนักงาน', 'สาขา', 'ลูกค้า', 'เงิน', 'ถาม', 'บัตร', 'การ์ด', 'คลับ']
99 ['ลูกค้า', 'เงิน', 'เหมือน', 'เหมือน', 'บริการ', 'ใบ', 'พูดจา', 'เต็มใจ', 'กำกับ']
107 ['ลูกค้า', 'ทำ', 'เงิน', 'เงิน', 'เงิน

### Normalized